In [ ]:
source=raw_input("Source of the data (GA or OWOX)?")
fullTableName=raw_input("Table Name")
start=raw_input("Start in format yyyymmdd")
end=raw_input("Start in format yyyymmdd")
cond=raw_input("list constraints using ; separation")
cond=cond.split(";")
if (cond==''):
    my_string='1=1'
else: 
    my_string=' AND '.join(cond)
if (source=="GA"):
    sql="with meow as (select visitid ,max(hits.time)/1000 as duration ,max(totals.pageviews) as pageviews ,max(totals.bounces) as bounce ,max(totals.totaltransactionrevenue)/1000000 as revenue ,max(totals.transactions) as cnt_transactions from `{}.ga_sessions_*` ,unnest(hits) as hits where 1=1 and _table_suffix between '{}' and '{}' group by visitid) select fullvisitorid ,a.visitid ,totals.newvisits ,date ,b.duration ,b.pageviews ,b.bounce ,b.revenue ,b.cnt_transactions ,device.devicecategory ,device.operatingsystem ,device.operatingsystemversion ,geonetwork.country ,geonetwork.region ,trafficsource.campaign ,trafficsource.campaigncode ,trafficsource.source ,trafficsource.medium ,channelgrouping ,hits.type ,hits.eventinfo.eventcategory ,hits.eventinfo.eventaction ,hits.eventinfo.eventlabel ,hits.eventinfo.eventvalue ,hits.referer ,hits.page.pagepath ,customdimensions.index ,customdimensions.value ,hits.experiment.experimentid ,hits.experiment.experimentvariant /*,promotion.promocreative ,promotion.promoid ,promotion.promoname ,promotion.promoposition ,promotionactioninfo.promoisview ,promotionactioninfo.promoisclick*/ from `{}.ga_sessions_*` a ,unnest(hits) as hits ,unnest(hits.customdimensions) as customdimensions left join meow b on a.visitid=b.visitid /*,unnest(hits.promotion) as promotion ,unnest(hits.promotionactioninfo) as promotionactioninfo*/ where 1=1 and _table_suffix between '{}' and '{}' AND {}".format(fullTableName,start,end,fullTableName,start,end,my_string)
    print(sql)
elif (source=="OWOX"):
    sql="with meow as (select sessionid ,max(hits.time)/1000 as duration ,max(totals.pageviews) as pageviews ,sum(transaction.transactionrevenue) as revenue ,max(totals.transactions) as cnt_transactions ,max(totals.events) as events from `{}.owoxbi_sessions_*` ,unnest(hits) as hits where 1=1 and _table_suffix between '{}' and '{}' group by sessionid) select clientid ,a.sessionid ,newvisits ,date ,b.duration ,b.pageviews ,case when b.events<=1 then 1 else 0 end as bounce ,b.revenue ,b.cnt_transactions ,a.device.devicecategory ,a.device.operatingsystem ,a.device.operatingsystemversion ,geonetwork.country ,geonetwork.region ,trafficsource.campaign ,trafficsource.source ,trafficsource.medium ,trafficsource.channelgrouping ,hits.type ,hits.pagepath ,landingpage ,hits.eventinfo.eventcategory ,hits.eventinfo.eventaction ,hits.eventinfo.eventlabel ,hits.eventinfo.eventvalue ,hits.referer ,customdimensions.index ,customdimensions.value ,experiment.experimentid ,experiment.experimentvariant /*,hits.promotion.promocreative ,hits.promotion.promoid ,hits.promotion.promoname ,hits.promotion.promoposition ,hits.promotionactioninfo*/ from `{}.owoxbi_sessions_*` a left join meow b on a.sessionid=b.sessionid ,unnest(hits) as hits ,unnest(hits.customdimensions) as customdimensions where 1=1 and _table_suffix between '{}' and '{}' AND {}".format(fullTableName,start,end,fullTableName,start,end,my_string)
    print(sql)
else:
    print("Invalid source")
